In [ ]:
from numcosmo_py import Ncm, Nc
import sys
import math
import matplotlib.pyplot as plt
import numpy as np
import warnings
from matplotlib.pyplot import cm

from scipy.integrate import odeint
from scipy.interpolate import interp1d
from numcosmo_py.plotting.tools import set_rc_params_article, latex_float

In [ ]:
__name__ = "NcContext"

Ncm.cfg_init ()
Ncm.cfg_set_log_handler (lambda msg: sys.stdout.write (msg) and sys.stdout.flush ())

In [ ]:
set_rc_params_article(ncol=2)

In [ ]:
qgw = Nc.HICosmoQGW.new()
adiab = Nc.HIPertAdiab.new()

In [ ]:
qgw.props.xb = 1.0e35
qgw.props.w = 1.0e-5

In [ ]:

tAi = -200.0
tAf = -5.0e-2

ki = 1.0e0
kf = 1.0e2
k_a = np.geomspace(ki, kf, 1000)

adiab.set_k(1.0e-1)
adiab.set_tf(60.0)
adiab.set_reltol(1.0e-11)
adiab.set_max_order_2(False)

Found, tAD = adiab.find_adiab_time_limit(qgw, tAi, tAf, 1.0e-3)

print(f"Found: {Found}, tAD: {tAD}")

In [ ]:
adiab.set_init_cond_adiab(qgw, tAD)
adiab.prepare(qgw)

In [ ]:
ta, ts = adiab.get_time_array()

In [ ]:
Pzeta = [adiab.eval_powspec_zeta_at(qgw, t) for t in ta]
PPsi = [adiab.eval_powspec_Psi_at(qgw, t) for t in ta]
Pdrho = [adiab.eval_powspec_drho_at(qgw, t) for t in ta]

In [ ]:
fig = plt.figure()

plt.plot(ta, Pzeta)
plt.plot(ta, PPsi)
plt.plot(ta, Pdrho)

plt.yscale('log')

plt.grid()

In [ ]:
ka = np.geomspace(1.0e-1, 1.0e10, 10000)

In [ ]:
Pzetak = adiab.eval_powspec_zeta(qgw, -300.0, -1.0, 1.0e-7, ka, 10.0)
PPsik = adiab.eval_powspec_Psi(qgw, -300.0, -1.0, 1.0e-7, ka, 10.0)
Pdrhok = adiab.eval_powspec_drho(qgw, -300.0, -1.0, 1.0e-7, ka, 10.0)

In [ ]:
fig = plt.figure()

plt.plot(ka, Pzetak)
plt.xscale('log')
plt.yscale('log')

In [ ]:
fig = plt.figure()

plt.plot(ka, PPsik)
plt.xscale('log')
plt.yscale('log')

In [ ]:
fig = plt.figure()

plt.plot(ka, Pdrhok)
plt.xscale('log')
plt.yscale('log')

In [ ]:
print(f"{np.polyfit(np.log(ka), np.log(Pzetak), 1)[0]} "
      f"{np.polyfit(np.log(ka), np.log(Pzetak), 1)[0] / (12.0 * qgw.props.w / (1.0 + 3.0 * qgw.props.w))}"
)
print(f"{np.polyfit(np.log(ka), np.log(Pdrhok), 1)[0]}")
print(f"{np.polyfit(np.log(ka), np.log(PPsik), 1)[0]}")

In [ ]:
ps = Nc.PowspecMLFixSpline()
Pdrhok_s = Ncm.SplineCubicNotaknot()
Pdrhok_s.set_array(ka, Pzetak, True)
ps.set_spline(Pdrhok_s)
ps.set_kmin(ka[0])
ps.set_kmax(ka[-1])

In [ ]:
psf = Ncm.PowspecFilter.new(ps, Ncm.PowspecFilterType.TOPHAT)
psf.set_best_lnr0()

In [ ]:
psf.prepare(qgw)

In [ ]:
psf.eval_sigma(0.0, 1.0)

In [ ]:
r_a = np.geomspace(psf.get_r_min(), psf.get_r_max(), 1000)
sigma_r = [psf.eval_sigma(0.0, r) for r in r_a]

In [ ]:
fig = plt.figure()

plt.plot(r_a, sigma_r)
plt.xscale('log')
plt.yscale('log')